In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchtext==0.10.0 -f https://download.pytorch.org/whl/cu111/torch_stable.html
!pip install transformers
!pip install pytorch_lightning
!pip install sentencepiece

     |████████████████████████████████| 149.9 MB 25 kB/s 
     |████████████████████████████████| 57 kB 2.7 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.10
    Uninstalling google-api-python-client-1.12.10:
      Successfully uninstalled google-api-python-client-1.12.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.300 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
     |█████████████                   | 834.1 MB 1.2 MB/s eta 0:16:55tcmalloc: large alloc 1147494400 bytes == 0x56021e10c000 @  0x7fbd133ea615 0x56021b4123bc 0x56021b4f318a 0x56021b4151cd 0x56021b507b3d 0x56021b489458 0x56021b48402f 0x56021b416aba 0x56021b4892c0 0x560

     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [ ]:
!pip list | grep torch

pytorch-lightning             1.5.10
torch                         1.9.0+cu111
torch-xla                     1.9
torchaudio                    0.10.0+cu111
torchmetrics                  0.7.2
torchsummary                  1.5.1
torchtext                     0.10.0
torchvision                   0.10.0+cu111


In [ ]:
import pickle
import pandas
import numpy as np
from transformers import T5Model, T5Tokenizer, AdamW
from torch.optim import AdamW
import pandas as pd
import torch
from torch.utils.data import Dataset , DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
torch.cuda.memory_allocated()

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/xla_device.py", line 32, in inner_f
    queue.put(func(*args, **kwargs))
  File "/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/xla_device.py", line 73, in _is_device_tpu
    return (xm.xrt_world_size() > 1) or bool(xm.get_xla_supported_devices("TPU"))
  File "/usr/local/lib/python3.7/dist-packages/torch_xla/core/xla_model.py", line 136, in get_xla_supported_devices
    xla_devices = _DEVICES.value
  File "/usr/local/lib/python3.7/dist-packages/torch_xla/utils/utils.py", line 32, in value
    self._value = self._gen_fn()
  File "/usr/local/lib/python3.7/dist-packages/torch_xla/core/xla_model.py", line 18, in <lambda>
    _DEVICES = xu.LazyProperty(lambda: torch_xla._XLAC._xla_get_devices())
RuntimeError: tensorflow/compiler/xla/xla_client/computation_client.cc:273 : Missing XLA configuration


0

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
root = "drive/MyDrive/lign167_data"

In [ ]:
from transformers import T5ForConditionalGeneration

t5 = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [ ]:
romantic_fileList = []
with open(f"{root}/humor_train.p", mode="rb") as f:
    romantic_fileList = pickle.load(f)
humor_fileList = []
with open(f"{root}/humor_train.p", mode="rb") as f:
    humor_fileList = pickle.load(f)
romantic_captions = []

In [ ]:
original_df = pandas.read_csv(f"{root}/Flickr8k.token.txt", sep="\t", header=None, names=["id", "sentence"])
original_df = original_df.assign(id=original_df.id.str.split("#").str[0])
# temporarily only use the first caption
# original_df = original_df.groupby("id").agg(lambda x: x.iloc[np.random.randint(5)])
romantic_df = pandas.read_csv(f"{root}/romantic_train.txt", names=["sentence"]).assign(id=romantic_fileList)
humor_df = pandas.read_csv(f"{root}/funny_train.txt", names=["sentence"]).assign(id=humor_fileList)
original_df = original_df.merge(romantic_df, on="id", how="right", suffixes=["_original", "_romantic"]).merge(humor_df, on="id", how="right")
df = original_df.rename(columns={"sentence": "sentence_humor"})
df = df.dropna()
df_romantic = df.assign(original=df.sentence_original.apply(lambda s: "Romantic: " + s), transferred=df.sentence_romantic)\
                .get(["original", "transferred"])
df_funny = df.assign(original=df.sentence_original.apply(lambda s: "Funny: " + s), transferred=df.sentence_humor)\
                .get(["original", "transferred"])
df = pd.concat([df_romantic, df_funny])
df.get("transferred").apply(str.split).apply(len).describe()

In [ ]:
class FlickrDataset(Dataset):
    def __init__(self,original,transferred):
        self.original = original
        self.transferred = transferred

    def __len__(self):
        return len(self.original)

    def to_token(self,sentence):
        return tokenizer.encode(sentence, max_length=32, truncation=True, padding="max_length", return_tensors="pt")

    def __getitem__(self,index):
        original_token = self.to_token(self.original.iloc[index])
        transferred_token = self.to_token(self.transferred.iloc[index])
        return original_token, transferred_token

In [ ]:
dataset = FlickrDataset(df.original,df.transferred)

In [ ]:
train_dataset, val_dataset = train_test_split(df, test_size=0.2)
batch_size = 32

In [ ]:
class FlickrDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.train = FlickrDataset(train_dataset.original,train_dataset.transferred)
        self.test = FlickrDataset(val_dataset.original,val_dataset.transferred)
        self.val = FlickrDataset(val_dataset.original,val_dataset.transferred)

    def train_dataloader(self):
        return DataLoader(self.train , batch_size = batch_size , shuffle = True, num_workers=4)
    def test_dataloader(self):
        return DataLoader(self.test , batch_size = batch_size , shuffle = False, num_workers=4)
    def val_dataloader(self):
        return DataLoader(self.val , batch_size = batch_size , shuffle = False, num_workers=4)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# next(iter(FlickrDataModule().train_dataloader()))[0].shape
next(iter(FlickrDataModule().val_dataloader()))[0].shape


NameError: ignored

In [ ]:
class StyleTransferModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        t5.train()
        self.neural_net = t5

    def forward(self, original, transferred):
        original = original.view(original.shape[0], -1)
        transferred = transferred.view(transferred.shape[0], -1)
        return self.neural_net(input_ids=original, labels=transferred)

    def configure_optimizers(self):
        return AdamW(self.parameters(), 1e-4)

    def training_step(self,batch,batch_idx):
        original, transferred = batch[0], batch[1]
        output = self(original, transferred)
        return output.loss

    def test_step(self,batch,batch_idx):
        original, transferred = batch[0], batch[1]
        output = self(original, transferred)
        return output.loss

    def validation_step(self,batch,batch_idx):
        original, transferred = batch[0], batch[1]
        output = self(original, transferred)
        self.log("val_loss", output.loss)
        return output.loss

In [ ]:
from pytorch_lightning import Trainer
model = StyleTransferModel()
module = FlickrDataModule()

checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_loss")

trainer = Trainer(max_epochs = 10, tpu_cores=8, default_root_dir=root, callbacks=[checkpoint_callback], precision=16)
trainer.fit(model,module)

In [ ]:
# orig_sample = val_dataset.sample()["original"].iloc[0]
orig_sample = "A man is biking down really rough terrain ."
print(orig_sample)
input_ids = tokenizer(orig_sample, return_tensors="pt").input_ids

outputs = model.neural_net.generate(input_ids, max_length=64)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))